In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers, losses
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

### Load the data

In [3]:
tfds.image_classification.cats_vs_dogs._URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
(train_ds, test_ds), metadata = tfds.load('cats_vs_dogs', split=['train[:35%]', 'train[90%:95%]'],
                                          with_info=True, as_supervised=True, shuffle_files=True,)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteVORQMP/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [4]:
# info related to the dataset
metadata

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.0',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_path=PosixGPath('/tmp/tmp0svgytmttfds'),
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=689.64 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=8>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
  

### More about the data

In [5]:
tot_train_images = train_ds.__len__().numpy()
tot_test_images = test_ds.__len__().numpy()

print(f"The total number of training images is: {tot_train_images}")
print(f"The total number of test images is: {tot_test_images}")
im_iter = train_ds.__iter__()

The total number of training images is: 8142
The total number of test images is: 1163


In [6]:
# visualize an image in the training dataset
# image, label = next(im_iter)
# plt.figure(figsize=(8,6))
# label = (1-label.numpy())*"Cat" + label.numpy()*"Dog"
# plt.imshow(image.numpy())
# plt.title(label)
# print(f"The original shape of this image is: {image.numpy().shape}")

As you can see by running multiple times the code cell above, the images in the dataset have different shapes (i.e. different number of pixels)

We need to resize the images to a common shape

### Pre-process the data

In [7]:
def image_formatting(image, label):
    # converting pixel values (uint8) to float32 type
    image = tf.cast(image, tf.float32)
    # normalizing the data to be in range of -1, +1
    image = applications.resnet_v2.preprocess_input(image)
    # resizing all images to a shape of 224x*224*3
    image = tf.image.resize(image, (224, 224))
    return image, label

# converting all images to same shape and formatting them for quicker training of the model
train = train_ds.map(image_formatting)
test = test_ds.map(image_formatting)

In [8]:
# suffle the data abd create batches
batch_size = 128
shuffle_size = 1000

train_batches = train.shuffle(shuffle_size).batch(batch_size)
test_batches = test.batch(batch_size)

### Create a pre-trained model

We will exploit keras.applications API to create a pre-trained model from ResNet152V2. This is a very deep convolutional neural network with ~60 million parameters that was trained using the ImageNet dataset. We will not re-train the entire network on the Cats_vs_Dog data (This would be computationally demanding). Instead, we will keep the majority of the network by replacing only the last fully connected layers with new layers that we are going to train on our data. Basically, ResNet152V2 will be used for feature extraction.

keras.applications: https://keras.io/api/applications/
ResNet152V2: https://keras.io/api/applications/resnet/#resnet152v2-function

In [9]:
# Pre-trained model
base_model = applications.ResNet152V2(
    weights='imagenet',  # Load weights pre-trained on the ImageNet dataset
    input_shape=(224, 224, 3),
    include_top=False,  # do not include the classifier at the top
)
base_model.trainable = False  # do not train the model

model = models.Sequential([
    base_model,
    layers.GlobalAvgPool2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model.summary()

234545216/234545216 [==============================] - 11s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 7, 7, 2048)        58331648  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 58,856,449
Trainable params: 524,801
Non-trainable params: 58,331,648
_________________________________________________________________


In [10]:
## To-do
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_batches, workers=32, batch_size=128, epochs=5, validation_data=test_batches)


Epoch 1/5
64/64 [==============================] - 93s 1s/step - loss: 0.0605 - accuracy: 0.9807 - val_loss: 0.0396 - val_accuracy: 0.9914
Epoch 2/5
64/64 [==============================] - 69s 1s/step - loss: 0.0208 - accuracy: 0.9937 - val_loss: 0.0342 - val_accuracy: 0.9914
Epoch 3/5
64/64 [==============================] - 65s 991ms/step - loss: 0.0087 - accuracy: 0.9972 - val_loss: 0.0354 - val_accuracy: 0.9914
Epoch 4/5
64/64 [==============================] - 65s 990ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0402 - val_accuracy: 0.9897
Epoch 5/5
64/64 [==============================] - 66s 1s/step - loss: 0.0023 - accuracy: 0.9996 - val_loss: 0.0396 - val_accuracy: 0.9914
